In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
arr = np.array

In [4]:
dtype = {'trip_id': np.str,
         'end_station': 'category',
         'start_station': 'category',
         'trip_route_category': 'category',
         'passholder_type': 'category',
         'bike_id': np.str,
        }
rides = pd.read_csv('../data/LABikeData.csv', dtype=dtype, parse_dates=['start_time', 'end_time'], infer_datetime_format=True)
names=['station', 'station_name', 'go_live_date', 'region', 'status']
dtype=dict(zip(names, ['category', np.str, np.str, np.str, np.str]))
stations = pd.read_csv('../data/Station_Table.csv', skiprows=1, names=names, dtype=dtype, parse_dates=['go_live_date'], infer_datetime_format=True)

In [5]:
df2.go_live_date = pd.to_datetime(df2.go_live_date)
merged = pd.merge(df, df2.add_prefix('start_'), on='start_station', how='left')
merged = pd.merge(merged, df2.add_prefix('end_'), on='end_station', how='left')
dtla = merged[merged.start_region == 'DTLA']

NameError: name 'df2' is not defined

In [ ]:
rides.dataframe() \
    .sort_values(by="start_time",ascending=True) \
    .set_index("start_time")
    .last("5M")

In [12]:
stations = pd.read_csv("suggested_locations.csv", dtype={'id': 'category'})
comments = pd.read_csv("suggested_comments.csv", dtype={'id': 'category'})

In [13]:
ncomments = comments.groupby('id').id.count().rename('ncomments') + 1
newdf = stations.join(ncomments, on='id')
newdf.ncomments = locations.ncomments.fillna(1)
comments_by_station = comments.groupby('id').comment.apply(list).rename('comments')
# sns.distplot(ncomments)

In [17]:
from itertools import chain
locations = newdf.join(comments_by_station, on='id').\
                groupby(['lat', 'lng']).\
                agg({
                    'id': 'count',
                    'ncomments': 'sum',
                    'comment': lambda x: ''.join(x.dropna()),
                    'comments': lambda x: list(chain(*x.dropna()))
                }).reset_index().rename(columns={'id': 'nstations'})
locations.comments = locations.apply(lambda r: r.comments + [r.comment], axis=1)
locations = locations.drop('comment', axis=1)

In [ ]:
locations[['lat', 'lng' , 'comments', 'ncomments']].
    rename(columns={'ncomments':'weight'}).
    to_json('latlng.json', orient='records')